# Random Forest algorithmn with GEE - Accuracy Assesment

To access the accuracy of a classifier, use a `ConfusionMatrix`.

### Import libraries

In [1]:
import ee
import geemap

### Create an interactive map

In [2]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Add data to the map

Let's add the [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD), which can be used to create training data with class labels. 

![](https://i.imgur.com/7QoRXxu.png)

In [3]:
NLCD2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
Map.addLayer(NLCD2016, {}, 'NLCD 2016')

Load the NLCD metadata to find out the Landsat image IDs used to generate the land cover data. 

In [4]:
NLCD_metadata = ee.FeatureCollection("users/giswqs/landcover/NLCD2016_metadata")
Map.addLayer(NLCD_metadata, {}, 'NLCD Metadata')

In [5]:
point = ee.Geometry.Point([-88.3070, 41.7471])  # Chicago

In [6]:
metadata = NLCD_metadata.filterBounds(point).first()
region = metadata.geometry()

In [7]:
metadata.get('2016on_bas').getInfo()

'LC08_2016256'

In [8]:
doy = metadata.get('2016on_bas').getInfo().replace('LC08_', '')
doy

'2016256'

In [9]:
ee.Date.parse('YYYYDDD', doy).format('YYYY-MM-dd').getInfo()

'2016-09-12'

In [10]:
start_date = ee.Date.parse('YYYYDDD', doy)
end_date = start_date.advance(1, 'day')

In [11]:
image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate(start_date, end_date)
    .first()
    .select('B[1-7]')
    .clip(region)
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [12]:
nlcd_raw = NLCD2016.clip(region)
Map.addLayer(nlcd_raw, {}, 'NLCD')

### Prepare for consecutive class labels

In this example, we are going to use the [USGS National Land Cover Database (NLCD)](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD) to create label dataset for training.

First, we need to use the `remap()` function to turn class labels into consecutive integers. 

In [13]:
raw_class_values = nlcd_raw.get('landcover_class_values').getInfo()
print(raw_class_values)

[11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]


In [14]:
n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [15]:
class_palette = nlcd_raw.get('landcover_class_palette').getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [16]:
nlcd = nlcd_raw.remap(raw_class_values, new_class_values).select(
    ['remapped'], ['landcover']
)
nlcd = nlcd.set('landcover_class_values', new_class_values)
nlcd = nlcd.set('landcover_class_palette', class_palette)

In [17]:
Map.addLayer(nlcd, {}, 'NLCD')
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

### Make training data

In [18]:
# Make the training dataset.
points = nlcd.sample(
    **{
        'region': region,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, 'training', False)

In [19]:
print(points.size().getInfo())

5000


In [20]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-88.56212124700772, 42.210469414463425]}, 'id': '0', 'properties': {'landcover': 17}}


### Split training and testing

In [21]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
sample = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

# Adds a column of deterministic pseudorandom numbers.
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [22]:
training.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '2_0',
 'properties': {'B1': 72,
  'B2': 215,
  'B3': 579,
  'B4': 543,
  'B5': 2558,
  'B6': 1782,
  'B7': 1103,
  'landcover': 3,
  'random': 0.18158720869622524}}

In [23]:
validation.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 174,
  'B2': 223,
  'B3': 614,
  'B4': 368,
  'B5': 4221,
  'B6': 1737,
  'B7': 766,
  'landcover': 17,
  'random': 0.889822614157854}}

### Train the classifier

In this examples, we will use random forest classification algorithm.

In [24]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

### Classify the image

In [25]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

### Render categorical map

To render a categorical map, we can set two image properties: `classification_class_values` and `classification_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps. 

In [26]:
class_values = nlcd.get('landcover_class_values').getInfo()
print(class_values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [27]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [28]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [29]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

### Add a legend to the map

In [30]:
Map.add_legend(builtin_legend='NLCD')
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

### Accuracy assessment

#### Training dataset

`confusionMatrix()` computes a 2D confusion matrix for a classifier based on its training data.

In [31]:
train_accuracy = classifier.confusionMatrix()

In [32]:
train_accuracy.getInfo()

[[272, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 183, 6, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 0],
 [0, 0, 1, 421, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0],
 [1, 0, 1, 15, 171, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 3, 8, 96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 2, 2, 0, 0, 0, 212, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 2, 0, 13, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 32, 0, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Overall Accuracy essentially tells us out of all of the reference sites what proportion were mapped correctly. The overall accuracy is usually expressed as a percent, with 100% accuracy being a perfect classification where all reference site were classified correctly.

In [33]:
train_accuracy.accuracy().getInfo()

0.9562445667922341

The `Kappa Coefficient` is generated from a statistical test to evaluate the accuracy of a classification. Kappa essentially evaluates how well the classification performed as compared to just randomly assigning values, i.e. did the classification do better than random. The Kappa Coefficient can range from -1 t0 1. A value of 0 indicated that the classification is no better than a random classification. A negative number indicates the classification is significantly worse than random. A value close to 1 indicates that the classification is significantly better than random. 

In [34]:
train_accuracy.kappa().getInfo()

0.9376575979043613

#### Validation dataset

In [35]:
validated = validation.classify(classifier)

In [36]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 174,
  'B2': 223,
  'B3': 614,
  'B4': 368,
  'B5': 4221,
  'B6': 1737,
  'B7': 766,
  'classification': 17,
  'landcover': 17,
  'random': 0.889822614157854}}

`errorMatrix` computes a 2D error matrix for a collection by comparing two columns of a collection: one containing the actual values, and one containing predicted values.

In [37]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [38]:
test_accuracy.getInfo()

[[145, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 13, 18, 2, 0, 0, 15, 0, 0, 0, 0, 1, 0, 0, 0, 2, 54, 1, 0],
 [2, 0, 8, 113, 20, 2, 1, 7, 0, 0, 0, 0, 1, 0, 0, 0, 3, 33, 3, 0],
 [1, 0, 1, 27, 34, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 1, 0],
 [0, 0, 1, 2, 18, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 12, 6, 1, 0, 0, 54, 0, 0, 0, 0, 0, 0, 0, 0, 1, 14, 5, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 10, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [39]:
test_accuracy.accuracy().getInfo()

0.7023886378308586

In [40]:
test_accuracy.kappa().getInfo()

0.5639057934062713

### Download confusion matrix
We download generated confusion matrix result into respective csvs

In [41]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())

with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

### Reclassify land cover map

In [42]:
landcover = landcover.remap(new_class_values, raw_class_values).select(
    ['remapped'], ['classification']
)

In [43]:
landcover = landcover.set('classification_class_values', raw_class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [44]:
Map.addLayer(landcover, {}, 'Final land cover')
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

### Export the result

Export the result directly to our computer:

In [45]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'landcover.tif')

In [46]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)

Generating URL ...
Please wait ...
Data downloaded to /home/ridhhi/Downloads/landcover.tif
